# Download full recordings

author: laquitainesteeve@gmail.com

purpose: download all full recordings from Dandi Archive

Execution time: 
- writing speed: maximize chunk_size, set to n_jobs=20. This maximizes speed while avoiding overhead [1].

Hardware: CPU

Tested on: 32 cores, 2TB storage, 188GB RAM Ubuntu machine




# Setup 

Activate virtual environment (envs/spikebias.yml)

```bash
python -m ipykernel install --user --name dandi --display-name "dandi"
```

In [1]:
%%time 

# import python packages
import os
import numpy as np
from time import time
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
import spikeinterface.sorters as ss
import spikeinterface
import uuid
from datetime import datetime
from dateutil.tz import tzlocal
from dandi.dandiapi import DandiAPIClient
import spikeinterface.extractors as se
print("spikeinterface", spikeinterface.__version__)

# set the project path
PROJ_PATH = "/home/steeve/steeve/epfl/code/spikebias"

# set the raw dataset path
RAW_DATASET = os.path.join(PROJ_PATH, "dataset/00_raw/")

spikeinterface 0.101.2
CPU times: user 533 ms, sys: 609 ms, total: 1.14 s
Wall time: 477 ms


/home/steeve/steeve/epfl/code/spikebias/envs/dandi/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom functions

In [2]:
class DataLoader:
    """Data loader for dandi datasets
    """
    def __init__(self, raw_dataset_path:str, dandiset_id:str, filepath:str, is_recording=True, is_sorting=True):
        self.raw_dataset_path = raw_dataset_path
        self.dandiset_id = dandiset_id
        self.filepath = filepath
        self.is_recording = is_recording
        self.is_sorting = is_sorting        
        self.recording = None
        self.sorting = None
        self.s3_path = None

    def load_data(self):
        
        # Get the file path on S3
        with DandiAPIClient() as client:
            asset = client.get_dandiset(self.dandiset_id, 'draft').get_asset_by_path(self.filepath)
            self.s3_path = asset.get_content_url(follow_redirects=1, strip_query=True)
        print("s3_path:", self.s3_path)

        # Get RecordingExtractor and SortingExtractor
        if self.is_recording:
            self.recording = se.NwbRecordingExtractor(file_path=self.s3_path, stream_mode="remfile")
        if self.is_sorting:
            self.sorting = se.NwbSortingExtractor(file_path=self.s3_path, stream_mode="remfile")

        # Report
        print('\nDownloaded recording:', self.recording)
        print('\nDownloaded sorting:', self.sorting)

    def save_data(self, recording_folder:str, sorting_folder:str, n_jobs=30, chunk_size=800000, dtype='float32', duration_secs=None):
        if self.is_recording:
            if duration_secs: 
                self.recording = self.recording.frame_slice(start_frame=0, end_frame=self.recording.sampling_frequency*duration_secs)
            self.recording.save(folder=recording_folder, n_jobs=n_jobs, verbose=True, progress_bar=True, overwrite=True, dtype=dtype, chunk_size=chunk_size)                

        if self.is_sorting:
            if duration_secs: 
                self.sorting = self.sorting.frame_slice(start_frame=0, end_frame=self.recording.sampling_frequency*duration_secs)
            self.sorting.save(folder=sorting_folder, progress_bar=True, overwrite=True)
        
        # Report
        print('\nSaved recording:', self.recording)
        print('\nSaved sorting:', self.sorting)            

## NPX spont biophy

- execution time: 50 min for 34:30 min
- storage: 117 GB (full 34.30 min)

In [9]:
%%time

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-001-fitted/sub-001-fitted_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_npx_spont")
sorting_folder = os.path.join(RAW_DATASET, "sorting_npx_spont")
#DURATION_SECS = 1800 # 30 min

# download and save dataset (full 3h40)
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
#data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32', duration_secs=DURATION_SECS) # save
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save full

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/0c3/2c4/0c32c475-1251-485b-9934-83667b3ba4ba

Downloaded recording: NwbRecordingExtractor: 384 channels - 40.0kHz - 1 segments - 82,319,958 samples 
                       2,058.00s (34.30 minutes) - float32 dtype - 117.76 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/0c3/2c4/0c32c475-1251-485b-9934-83667b3ba4ba

Downloaded sorting: NwbSortingExtractor: 1388 units - 1 segments - 40.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/0c3/2c4/0c32c475-1251-485b-9934-83667b3ba4ba
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=1.14 GiB - total_memory=34.33 GiB - chunk_duration=20.00s


write_binary_recording: 100%|██████████| 103/103 [50:11<00:00, 29.23s/it]  



Saved recording: NwbRecordingExtractor: 384 channels - 40.0kHz - 1 segments - 82,319,958 samples 
                       2,058.00s (34.30 minutes) - float32 dtype - 117.76 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/0c3/2c4/0c32c475-1251-485b-9934-83667b3ba4ba

Saved sorting: NwbSortingExtractor: 1388 units - 1 segments - 40.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/0c3/2c4/0c32c475-1251-485b-9934-83667b3ba4ba
CPU times: user 546 ms, sys: 261 ms, total: 807 ms
Wall time: 50min 22s


## NPX evoked biophy

- Execution time: 44 min for 60 min recording
- storage: 103 GB (full 1 hour)

In [8]:
%%time

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-002-fitted/sub-002-fitted_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_npx_evoked")
sorting_folder = os.path.join(RAW_DATASET, "sorting_npx_evoked")
#DURATION_SECS = 3600 # 60 min

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
#data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32', duration_secs=DURATION_SECS) # save
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save full

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

Downloaded recording: NwbRecordingExtractor: 384 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - float32 dtype - 103.51 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

Downloaded sorting: NwbSortingExtractor: 1836 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=1.14 GiB - total_memory=34.33 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 91/91 [43:38<00:00, 28.77s/it]   



Saved recording: NwbRecordingExtractor: 384 channels - 20.0kHz - 1 segments - 72,359,964 samples 
                       3,618.00s (1.00 hours) - float32 dtype - 103.51 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad

Saved sorting: NwbSortingExtractor: 1836 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/9d6/6ed/9d66ed40-af31-43aa-b4ba-246d2206dcad
CPU times: user 1.07 s, sys: 446 ms, total: 1.52 s
Wall time: 44min 5s


## Dense depth 1 biophy

Execution time: 62 mins

In [4]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-003-fitted/sub-003-fitted_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_dense_probe1")
sorting_folder = os.path.join(RAW_DATASET, "sorting_dense_probe1")

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/dec/e65/dece6568-cee4-4ade-80bf-c1166a03fe2a

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 34,299,965 samples 
                       1,715.00s (28.58 minutes) - float32 dtype - 16.36 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/dec/e65/dece6568-cee4-4ade-80bf-c1166a03fe2a

Downloaded sorting: NwbSortingExtractor: 287 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/dec/e65/dece6568-cee4-4ade-80bf-c1166a03fe2a
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=390.62 MiB - total_memory=11.44 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 43/43 [1:02:08<00:00, 86.72s/it]  



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 34,299,965 samples 
                       1,715.00s (28.58 minutes) - float32 dtype - 16.36 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/dec/e65/dece6568-cee4-4ade-80bf-c1166a03fe2a

Saved sorting: NwbSortingExtractor: 287 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/dec/e65/dece6568-cee4-4ade-80bf-c1166a03fe2a
CPU times: user 187 ms, sys: 128 ms, total: 315 ms
Wall time: 1h 2min 17s


## Dense depth 2 biophy

Execution time: 12 min

In [5]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-004-fitted/sub-004-fitted_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_dense_probe2")
sorting_folder = os.path.join(RAW_DATASET, "sorting_dense_probe2")

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/eef/9e9/eef9e95c-fb5b-46d2-a24c-878d8170b5e0

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 23,519,976 samples 
                       1,176.00s (19.60 minutes) - float32 dtype - 11.22 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/eef/9e9/eef9e95c-fb5b-46d2-a24c-878d8170b5e0

Downloaded sorting: NwbSortingExtractor: 770 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/eef/9e9/eef9e95c-fb5b-46d2-a24c-878d8170b5e0
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=390.62 MiB - total_memory=11.44 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 30/30 [12:42<00:00, 25.41s/it]  



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 23,519,976 samples 
                       1,176.00s (19.60 minutes) - float32 dtype - 11.22 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/eef/9e9/eef9e95c-fb5b-46d2-a24c-878d8170b5e0

Saved sorting: NwbSortingExtractor: 770 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/eef/9e9/eef9e95c-fb5b-46d2-a24c-878d8170b5e0
CPU times: user 255 ms, sys: 107 ms, total: 363 ms
Wall time: 12min 49s


## Dense depth 3 biophy

Execution time: 26 min

In [6]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-005-fitted/sub-005-fitted_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_dense_probe3")
sorting_folder = os.path.join(RAW_DATASET, "sorting_dense_probe3")

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/ee2/816/ee2816de-d861-4b55-9cde-416a52e54049

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 35,279,964 samples 
                       1,764.00s (29.40 minutes) - float32 dtype - 16.82 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/ee2/816/ee2816de-d861-4b55-9cde-416a52e54049

Downloaded sorting: NwbSortingExtractor: 1123 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/ee2/816/ee2816de-d861-4b55-9cde-416a52e54049
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=390.62 MiB - total_memory=11.44 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 45/45 [26:28<00:00, 35.29s/it]  



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 35,279,964 samples 
                       1,764.00s (29.40 minutes) - float32 dtype - 16.82 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/ee2/816/ee2816de-d861-4b55-9cde-416a52e54049

Saved sorting: NwbSortingExtractor: 1123 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/ee2/816/ee2816de-d861-4b55-9cde-416a52e54049
CPU times: user 428 ms, sys: 115 ms, total: 543 ms
Wall time: 26min 35s


## Marques-Smith

- Execution time: 10 min (for full 20.25 min)
- Storage: 26 GB

In [12]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-vivo-marques-smith/sub-vivo-marques-smith_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_marques_smith")
sorting_folder = None

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath, is_sorting=False)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/109/db6/109db6a7-500b-4e59-83ca-8422c27137cf

Downloaded recording: NwbRecordingExtractor: 384 channels - 30.0kHz - 1 segments - 36,451,538 samples 
                       1,215.05s (20.25 minutes) - int16 dtype - 26.07 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/109/db6/109db6a7-500b-4e59-83ca-8422c27137cf

Downloaded sorting: None
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=585.94 MiB - total_memory=17.17 GiB - chunk_duration=26.67s


write_binary_recording: 100%|██████████| 46/46 [09:56<00:00, 12.97s/it]  



Saved recording: NwbRecordingExtractor: 384 channels - 30.0kHz - 1 segments - 36,451,538 samples 
                       1,215.05s (20.25 minutes) - int16 dtype - 26.07 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/109/db6/109db6a7-500b-4e59-83ca-8422c27137cf

Saved sorting: None
CPU times: user 120 ms, sys: 154 ms, total: 275 ms
Wall time: 10min


## Horvath depth 1

execution time: 21 min

In [7]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-vivo-horvath-depth-1/sub-vivo-horvath-depth-1_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_horvath_probe1")
sorting_folder = None

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath, is_sorting=False)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/10a/c1a/10ac1a40-6918-4eea-b80c-d887bad92ae9

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 72,131,040 samples 
                       3,606.55s (1.00 hours) - int16 dtype - 17.20 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/10a/c1a/10ac1a40-6918-4eea-b80c-d887bad92ae9

Downloaded sorting: None
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=195.31 MiB - total_memory=5.72 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 91/91 [21:29<00:00, 14.17s/it]  



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 72,131,040 samples 
                       3,606.55s (1.00 hours) - int16 dtype - 17.20 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/10a/c1a/10ac1a40-6918-4eea-b80c-d887bad92ae9

Saved sorting: None
CPU times: user 124 ms, sys: 126 ms, total: 251 ms
Wall time: 21min 32s


## Horvath depth 2

-Execution time: 56 min

In [8]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-vivo-horvath-depth-2/sub-vivo-horvath-depth-2_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_horvath_probe2")
sorting_folder = None

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath, is_sorting=False)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/4d2/875/4d2875ba-d83b-44d5-9036-74f42e19e8a0

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 73,773,360 samples 
                       3,688.67s (1.02 hours) - int16 dtype - 17.59 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/4d2/875/4d2875ba-d83b-44d5-9036-74f42e19e8a0

Downloaded sorting: None
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=195.31 MiB - total_memory=5.72 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 93/93 [56:00<00:00, 36.13s/it]   



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 73,773,360 samples 
                       3,688.67s (1.02 hours) - int16 dtype - 17.59 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/4d2/875/4d2875ba-d83b-44d5-9036-74f42e19e8a0

Saved sorting: None
CPU times: user 129 ms, sys: 120 ms, total: 250 ms
Wall time: 56min 3s


## Horvath depth 3

Execution time: 11 min

In [3]:
%%time 

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-vivo-horvath-depth-3/sub-vivo-horvath-depth-3_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_horvath_probe3")
sorting_folder = None

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath, is_sorting=False)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/14a/205/14a205ea-306f-47fd-97e8-284a6f00626b

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 72,061,920 samples 
                       3,603.10s (1.00 hours) - int16 dtype - 17.18 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/14a/205/14a205ea-306f-47fd-97e8-284a6f00626b

Downloaded sorting: None
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=195.31 MiB - total_memory=5.72 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 91/91 [11:38<00:00,  7.68s/it]  



Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 72,061,920 samples 
                       3,603.10s (1.00 hours) - int16 dtype - 17.18 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/14a/205/14a205ea-306f-47fd-97e8-284a6f00626b

Saved sorting: None
CPU times: user 106 ms, sys: 146 ms, total: 252 ms
Wall time: 11min 43s


### Get raw data


In [ ]:
%%time 

# set dataset parameters
dandiset_id = '000034'
filepath = 'sub-MEAREC-250neuron-Neuropixels/sub-MEAREC-250neuron-Neuropixels_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_buccino")
sorting_folder = os.path.join(RAW_DATASET, "sorting_buccino")

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/465/9c0/4659c033-b336-4b8b-b947-77434cebf494

 NwbRecordingExtractor: 384 channels - 32.0kHz - 1 segments - 19,200,000 samples 
                       600.00s (10.00 minutes) - float32 dtype - 27.47 GiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/465/9c0/4659c033-b336-4b8b-b947-77434cebf494

 NwbSortingExtractor: 250 units - 1 segments - 32.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/465/9c0/4659c033-b336-4b8b-b947-77434cebf494
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=1.14 GiB - total_memory=34.33 GiB - chunk_duration=25.00s


write_binary_recording: 100%|██████████| 24/24 [10:01:15<00:00, 1503.14s/it]   


CPU times: user 1.52 s, sys: 448 ms, total: 1.96 s
Wall time: 10h 1min 27s


## Single-cell isolated traces with Reyes probe 

- 40 second recording
- Execution time: 4 min
- size: 390.62 MB 

In [4]:
%%time

# set dataset parameters
dandiset_id = '001250'
filepath = 'sub-biophy-isolated-traces-reyes/sub-biophy-isolated-traces-reyes_ses-006_ecephys.nwb'
recording_folder = os.path.join(RAW_DATASET, "recording_reyes_isolated_traces")
sorting_folder = os.path.join(RAW_DATASET, "sorting_reyes_isolated_traces")

# download and save dataset
data_loader = DataLoader(raw_dataset_path=RAW_DATASET, dandiset_id=dandiset_id, filepath=filepath)
data_loader.load_data() # Load the data
data_loader.save_data(recording_folder=recording_folder, sorting_folder=sorting_folder, n_jobs=30, chunk_size=800000, dtype='float32') # save

s3_path: https://dandiarchive.s3.amazonaws.com/blobs/2f3/edc/2f3edc58-b09e-4571-bb1c-1ffb2f768b57

Downloaded recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 799,900 samples - 39.99s 
                       float32 dtype - 390.58 MiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/2f3/edc/2f3edc58-b09e-4571-bb1c-1ffb2f768b57

Downloaded sorting: NwbSortingExtractor: 1 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/2f3/edc/2f3edc58-b09e-4571-bb1c-1ffb2f768b57
write_binary_recording 
n_jobs=30 - samples_per_chunk=800,000 - chunk_memory=390.62 MiB - total_memory=11.44 GiB - chunk_duration=40.00s


write_binary_recording: 100%|██████████| 1/1 [04:27<00:00, 267.91s/it]


Saved recording: NwbRecordingExtractor: 128 channels - 20.0kHz - 1 segments - 799,900 samples - 39.99s 
                       float32 dtype - 390.58 MiB
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/2f3/edc/2f3edc58-b09e-4571-bb1c-1ffb2f768b57

Saved sorting: NwbSortingExtractor: 1 units - 1 segments - 20.0kHz
  file_path: https://dandiarchive.s3.amazonaws.com/blobs/2f3/edc/2f3edc58-b09e-4571-bb1c-1ffb2f768b57
CPU times: user 129 ms, sys: 33.8 ms, total: 163 ms
Wall time: 4min 33s


# References

[1] https://github.com/SpikeInterface/spikeinterface/issues/3252
* effect of n_jobs and chunk_size on writing speed: